In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'NLP-with-Disaster-Tweets-cleaning-data-V3'
device = 'cuda'

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence.lower())

In [3]:
tokenize("@100")

['@', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(words))
    for idx,w in enumerate(words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [8]:
data = pd.read_csv('./data.csv').sample(frac=1)

In [9]:
X = data['text']

In [10]:
data

,id,keyword,location,text,target
1978,2846,cyclone,Geneva,Blending the old with the new in # Vanuatu to...,0
1800,2587,crash,NaN,? One night and we are gonna come and crash t...,0
2809,4041,disaster,"Calgary, AB",The @ rbcinsurance quote website = disaster...,0
6521,9325,survive,? icon by @Hashiren_3 ?,@ mochichiiiii @ hikagezero IT ' S IMPOSSIBL...,0
2371,3410,derail,"Los Angeles, CA",@ BV Bloomberg will publish anything negative...,0
...,...,...,...,...,...
6016,8591,screams,texas,SCREAMS AT MY OWN MOTHER,0
6008,8580,screams,lost in history,SCREAMS . WHERE IS EVERYONE . \n\noh wait sch...,0
6860,9832,trauma,"Little Rock, AR",@ thetimepast @ saalon I have childhood trau...,0
3280,4706,epicentre,Africa,RT @ calestous : Tanzania elephant populatio...,1


In [11]:
y = data['target']

In [12]:
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [13]:
for label in y.tolist():
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [14]:
labels

{0: 1, 1: 2}

In [15]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

7613it [00:02, 3243.99it/s]


In [16]:
words = sorted(set(words))

In [17]:
np.random.shuffle(data)

In [18]:
X = []
y = []

In [19]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

100%|██████████| 7613/7613 [00:21<00:00, 350.99it/s]


In [20]:
X[590]

array([0., 1., 0., ..., 0., 0., 0.])

In [21]:
y[0]

array([0., 1.])

In [22]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [23]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [24]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in zip(preds,y):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [25]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 1024
        self.activation = ReLU()
        self.linear1 = Linear(len(words),self.hidden)
        self.linear2 = Linear(self.hidden,self.hidden)
        self.linear3 = Linear(self.hidden,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.linear2(preds)
        preds = self.activation(preds)
        preds = self.linear3(preds)
        return preds

In [26]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [27]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()

wandb: Currently logged in as: ranuga-d (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


 57%|█████▋    | 57/100 [01:29<01:07,  1.57s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-15c1e5c8ae99>", line 17, in <module>
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
  File "<ipython-input-24-a2dadb1f2014>", line 6, in get_accuracy
    pred = int(torch.argmax(pred))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/indika/Sync/anaconda3/lib/python3.8/

TypeError: object of type 'NoneType' has no len()

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/media/indika/Sync/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/media/indika/Sync/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 149, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 125, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 978, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 799, in _communicate

Error in callback <function _WandbInit._pause_backend at 0x7f79dfb3ddc0> (for post_run_cell):


KeyboardInterrupt: 

In [ ]:
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(X,'X.pt')
torch.save(X,'X.pth')
torch.save(y,'y.pt')
torch.save(y,'y.pth')

In [ ]:
torch.save(X_train,'X_train.pt')
torch.save(X_test,'X_test.pth')
torch.save(y_train,'y_train.pt')
torch.save(y_test,'y_test.pth')

In [ ]:
torch.save(words,'words.pt')
torch.save(words,'words.pth')
torch.save(data,'data.pt')
torch.save(data,'data.pth')
torch.save(labels,'labels.pt')
torch.save(labels,'labels.pth')

In [ ]:
torch.save(idx,'idx.pt')
torch.save(idx,'idx.pth')

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/media/indika/Sync/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/media/indika/Sync/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 167, in check_status
    status_response = self._interface.communicate_stop_status()
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 114, in communicate_stop_status
    resp = self._communicate_stop_status(status)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 968, in _communicate_stop_status
    resp = self._communicate(req, local=True)
  File "/media/indika/Sync/anaconda3/lib/python3.8/site-packages/wandb/sdk/interface/interface.py", line 799, in _communicate
    return self._co